In [ ]:
pip install folium

In [ ]:
import pandas as pd
import folium
from folium.plugins import HeatMap
from google.colab import files


In [ ]:
uploaded=files.upload()

Saving AEGISDataset.csv to AEGISDataset.csv


In [ ]:
data_manila = pd.read_csv("AEGISDataset.csv")

In [ ]:
data_manila.head()

,lat,lon,flood_heig,elevation,precipitat
0,14.640394,121.055708,0,54.553295,9.0
1,14.698299,121.002132,0,21.856272,10.0
2,14.698858,121.100261,0,69.322807,16.0
3,14.571310,120.983334,0,10.987241,8.0
4,14.762232,121.075735,0,87.889847,18.0


In [ ]:
numeric_columns = ["lat","lon","flood_heig","elevation","precipitat"]
data_manila[numeric_columns] = data_manila[numeric_columns].apply(pd.to_numeric, errors ='coerce')
data_manila_clean = data_manila.dropna()
print("NaN after cleaning:")
print(data_manila_clean.isnull().sum())

NaN after cleaning:
lat           0
lon           0
flood_heig    0
elevation     0
precipitat    0
dtype: int64


In [ ]:
data_manila_clean["normalized_flood_heig"] = (data_manila_clean["flood_heig"]-data_manila_clean["flood_heig"].min())/(data_manila_clean["flood_heig"].max()-data_manila_clean["flood_heig"].min())
data_manila_clean["normalized_elevation"] = (data_manila_clean["elevation"]-data_manila_clean["elevation"].min())/(data_manila_clean["elevation"].max()-data_manila_clean["elevation"].min())
data_manila_clean["normalized_precipitat"] = (data_manila_clean["precipitat"]-data_manila_clean["precipitat"].min())/(data_manila_clean["precipitat"].max()-data_manila_clean["precipitat"].min())

In [ ]:
data_manila_clean["normalized_elevation"]=1-data_manila_clean["normalized_elevation"]

In [ ]:
data_manila_clean["risk_score"] =(data_manila_clean["normalized_flood_heig"] * 0.4 + data_manila_clean["normalized_elevation"] * 0.3 + data_manila_clean["normalized_precipitat"] * 0.3)

In [ ]:
print("\Risk Score for Each Location")
for index, row in data_manila_clean.iterrows():
    print(f"Latitude: {row['lat']}, Longtitude: {row['lon']}, Flood Height: {row['flood_heig']}, Risk Score: {row['risk_score']}")

\Risk Score for Each Location
Latitude: 14.64039374, Longtitude: 121.0557085, Flood Height: 0.0, Risk Score: 0.3067845962001993
Latitude: 14.69829935, Longtitude: 121.0021322, Flood Height: 0.0, Risk Score: 0.2142565956370926
Latitude: 14.69885841, Longtitude: 121.1002614, Flood Height: 0.0, Risk Score: 0.4550332224796582
Latitude: 14.57131006, Longtitude: 120.9833345, Flood Height: 0.0, Risk Score: 0.1501785229612331
Latitude: 14.7622323, Longtitude: 121.0757349, Flood Height: 0.0, Risk Score: 0.5442589325706005
Latitude: 14.6011784, Longtitude: 121.0885987, Flood Height: 0.0, Risk Score: 0.20439257482205014
Latitude: 14.70803089, Longtitude: 121.052674, Flood Height: 0.0, Risk Score: 0.31843511060499957
Latitude: 14.70920612, Longtitude: 121.0502922, Flood Height: 0.0, Risk Score: 0.3269265493680337
Latitude: 14.70962531, Longtitude: 121.0470059, Flood Height: 0.0, Risk Score: 0.3124979487075966
Latitude: 14.66933825, Longtitude: 121.1328981, Flood Height: 0.0, Risk Score: 0.40056450

In [ ]:
def color(risk_score):
    if risk_score < 0.2:
        return 'green'
    elif 0.2 <= risk_score < 0.4:
        return 'yellow'
    elif 0.4 <= risk_score < 0.6:
        return 'orange'
    elif 0.6 <= risk_score < 0.8:
        return 'red'
    else:
        return 'dark red'

data_manila_clean['risk_color'] = data_manila_clean['risk_score'].apply(color)

manila_cords = [14.599512, 120.984222]

flood_risk_map = folium.Map(location=manila_cords, zoom_start=10)

heat_data = data_manila_clean[['lat','lon','risk_score']].values.tolist()
HeatMap(heat_data, radius=15, max_zoom=13).add_to(flood_risk_map)

print(f"\nLatitude: {data_manila_clean.iloc[-1]['lat']}, Longitude: {data_manila_clean.iloc[-1]['lon']}, Flood Height: {data_manila_clean.iloc[-1]['flood_heig']}, Risk Score: {data_manila_clean.iloc[-1]['risk_score']}, Risk Color:{data_manila_clean.iloc[-1]['risk_color']}")
flood_risk_map.save('manila_flood_risk_map.html')
flood_risk_map


Latitude: 14.65701285, Longitude: 121.0544233, Flood Height: 8, Risk Score: 0.7032339707327798, Risk Color:red
